In [245]:
import arxiv
import urllib.request as libreq
import feedparser
import pdfminer.layout
import pdfminer.high_level
from io import StringIO
from pdfminer.layout import LAParams
from bs4 import BeautifulSoup as bs
import nltk
import re
import heapq
import boto3
import pdfminer3
import os
from pycontractions import Contractions
from PyPDF2 import PdfFileWriter, PdfFileReader #for deleting all images
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 
import numpy as np  
import pandas as pd   
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences  
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
ps = PorterStemmer() 

ModuleNotFoundError: No module named 'tensorflow'

## Downloading Breakdown

In [211]:
"""
python_arXiv_parsing_example.py

This sample script illustrates a basic arXiv api call
followed by parsing of the results using the 
feedparser python module.

Please see the documentation at 
http://export.arxiv.org/api_help/docs/user-manual.html
for more information, or email the arXiv api 
mailing list at arxiv-api@googlegroups.com.

urllib is included in the standard python library.
feedparser can be downloaded from http://feedparser.org/ .

Author: Julius B. Lucks

This is free software.  Feel free to do what you want
with it, but please play nice with the arXiv API!
"""

# Base api query url
base_url = 'http://export.arxiv.org/api/query?';

# Search parameters
search_query = 'cat:cs.LG' # search in the machine learning category
start = 10000                     # retreive the first 5 results
max_results = 5

query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                     start,
                                                     max_results)
#List of paper entries with all info
corpusEntry=[]
#Corresponding list of pdf download links 
corpusPDF=[]
#Corresponding list of Paper ID's
corpusID = []
#Corresponding list of Paper Abstracts
corpusAbstract = []
# Opensearch metadata such as totalResults, startIndex, 
# and itemsPerPage live in the opensearch namespase.
# Some entry metadata lives in the arXiv namespace.
# This is a hack to expose both of these namespaces in
# feedparser v4.1
feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'

# perform a GET request using the base_url and query
with libreq.urlopen(base_url+query) as url:
    response = url.read()

# parse the response using feedparser
feed = feedparser.parse(response)

# print out feed information
print ('Feed title: %s' % feed.feed.title)
print ('Feed last updated: %s' % feed.feed.updated)

# print opensearch metadata
print ('totalResults for this query: %s' % feed.feed.opensearch_totalresults)
print ('itemsPerPage for this query: %s' % feed.feed.opensearch_itemsperpage)
print ('startIndex for this query: %s'   % feed.feed.opensearch_startindex)

# Run through each entry, and print out information
for entry in feed.entries:
    corpusEntry.append(entry)
    print ('e-print metadata')
    print ('arxiv-id: %s' % entry.id.split('/abs/')[-1])
    corpusID.append(entry.id.split('/abs/')[-1])
    print ('Published: %s' % entry.published)
    print ('Title:  %s' % entry.title)
    
    # feedparser v4.1 only grabs the first author
    author_string = entry.author
    
    # grab the affiliation in <arxiv:affiliation> if present
    # - this will only grab the first affiliation encountered
    #   (the first affiliation for the first author)
    # Please email the list with a way to get all of this information!
    try:
        author_string += ' (%s)' % entry.arxiv_affiliation
    except AttributeError:
        pass
    
    print ('Last Author:  %s' % author_string)
    
    # feedparser v5.0.1 correctly handles multiple authors, print them all
    try:
        print ('Authors:  %s' % ', '.join(author.name for author in entry.authors))
    except AttributeError:
        pass

    # get the links to the abs page and pdf for this e-print
    for link in entry.links:
        if link.rel == 'alternate':
            print ('abs page link: %s' % link.href)
        elif link.title == 'pdf':
            
            corpusPDF.append({"pdf_url": link.href})
            print ('pdf link: %s' % link.href)
    
    # The journal reference, comments and primary_category sections live under 
    # the arxiv namespace
    try:
        journal_ref = entry.arxiv_journal_ref
    except AttributeError:
        journal_ref = 'No journal ref found'
    print ('Journal reference: %s' % journal_ref)
    
    try:
        comment = entry.arxiv_comment
    except AttributeError:
        comment = 'No comment found'
    print ('Comments: %s' % comment)
    
    # Since the <arxiv:primary_category> element has no data, only
    # attributes, feedparser does not store anything inside
    # entry.arxiv_primary_category
    # This is a dirty hack to get the primary_category, just take the
    # first element in entry.tags.  If anyone knows a better way to do
    # this, please email the list!
    print ('Primary Category: %s' % entry.tags[0]['term'])
    
    # Lets get all the categories
    all_categories = [t['term'] for t in entry.tags]
    print ('All Categories: %s' % (', ').join(all_categories))
    
    # The abstract is in the <summary> element
    print ('Abstract: %s' %  entry.summary)
    corpusAbstract.append(entry.summary)

Feed title: ArXiv Query: search_query=cat:cs.LG&id_list=&start=10000&max_results=5
Feed last updated: 2020-04-27T00:00:00-04:00
totalResults for this query: 54026
itemsPerPage for this query: 5
startIndex for this query: 10000
e-print metadata
arxiv-id: 1712.07449v2
Published: 2017-12-20T12:41:20Z
Title:  In silico generation of novel, drug-like chemical matter using the LSTM
  neural network
Last Author:  Valery Polyakov
Authors:  Peter Ertl, Richard Lewis, Eric Martin, Valery Polyakov
abs page link: http://arxiv.org/abs/1712.07449v2
pdf link: http://arxiv.org/pdf/1712.07449v2
Journal reference: No journal ref found
Comments: in this version fixed some reference numbers
Primary Category: cs.LG
All Categories: cs.LG, q-bio.QM
Abstract: The exploration of novel chemical spaces is one of the most important tasks
of cheminformatics when supporting the drug discovery process. Properly
designed and trained deep neural networks can provide a viable alternative to
brute-force de novo approach

In [212]:
corpusEntry #This is the full entry, can access id, link, when updated etc.

[{'arxiv_comment': 'in this version fixed some reference numbers',
  'arxiv_primary_category': {'scheme': 'http://arxiv.org/schemas/atom',
   'term': 'cs.LG'},
  'author': 'Valery Polyakov',
  'author_detail': {'name': 'Valery Polyakov'},
  'authors': [{'name': 'Peter Ertl'},
   {'name': 'Richard Lewis'},
   {'name': 'Eric Martin'},
   {'name': 'Valery Polyakov'}],
  'guidislink': True,
  'id': 'http://arxiv.org/abs/1712.07449v2',
  'link': 'http://arxiv.org/abs/1712.07449v2',
  'links': [{'href': 'http://arxiv.org/abs/1712.07449v2',
    'rel': 'alternate',
    'type': 'text/html'},
   {'href': 'http://arxiv.org/pdf/1712.07449v2',
    'rel': 'related',
    'title': 'pdf',
    'type': 'application/pdf'}],
  'published': '2017-12-20T12:41:20Z',
  'published_parsed': time.struct_time(tm_year=2017, tm_mon=12, tm_mday=20, tm_hour=12, tm_min=41, tm_sec=20, tm_wday=2, tm_yday=354, tm_isdst=0),
  'summary': 'The exploration of novel chemical spaces is one of the most important tasks\nof chemin

In [213]:
corpusPDF #This is the forced PDF link as a pdf_url item for the download function

[{'pdf_url': 'http://arxiv.org/pdf/1712.07449v2'},
 {'pdf_url': 'http://arxiv.org/pdf/1712.07454v1'},
 {'pdf_url': 'http://arxiv.org/pdf/1712.07525v1'},
 {'pdf_url': 'http://arxiv.org/pdf/1712.07570v1'},
 {'pdf_url': 'http://arxiv.org/pdf/1712.07628v1'}]

## Download

In [214]:
library = r'C:\Users\Al\Documents\ByteSizeArxiv\library'

In [215]:
# Override the default filename format by defining a slugify function. So can force pdf link for all even without listed
activePDF = arxiv.download(corpusPDF[1],library, slugify=lambda x: corpusEntry[1].get('id').split('/')[-1])

In [216]:
activePDF

'C:\\Users\\Al\\Documents\\ByteSizeArxiv\\library/1712.07454v1.pdf'

##  Read PDF

In [217]:
text=pdfminer.high_level.extract_text(activePDF, codec='utf-8', laparams=None)

In [218]:
text

'7\n1\n0\n2\n \nc\ne\nD\n \n0\n2\n \n \n]\nL\nM\n\n.\nt\na\nt\ns\n[\n \n \n1\nv\n4\n5\n4\n7\n0\n.\n2\n1\n7\n1\n:\nv\ni\nX\nr\na\n\nFast kNN mode seeking clustering\napplied to active learning\n\nRobert P.W. Duin\nPRLab, Delft University of Technology, Netherlands\nr.p.w.duin@tudelft.nl\n\nSergey Verzakov\nPrime Vision, Delft, Netherlands\ns.verzakov@gmail.com\n\nDecember 21, 2017\n\nAbstract\n\nA signiﬁcantly faster algorithm is presented for the original kNN mode\nseeking procedure. It has the advantages over the well-known mean\nshift algorithm that it is feasible in high-dimensional vector spaces\nand results in uniquely, well deﬁned modes. Moreover, without any\nadditional computational eﬀort it may yield a multi-scale hierarchy\nof clusterings. The time complexity is just O(n\nn). Resulting com-\nputing times range from seconds for 104 objects to minutes for 105\nobjects and to less than an hour for 106 objects. The space complex-\nity is just O(n). The procedure is well suited fo

## Clean Text

In [219]:
#Clean the Text; start from most to least specific
cleanedText = text
cleanedText = cleanedText.split("Abstract",1)[1]#Removes all junk before abstract
cleanedText = cleanedText.rsplit("\nReferences\n", 1)[0] #Removes all references, starts from back
cleanedText = re.sub(r'\x0c','', cleanedText) #Remove page breaks
cleanedText = re.sub(r'-\n','', cleanedText)
cleanedText = re.sub(r'\n-','', cleanedText) #Hyphens before & after new lines are usually added for continuation of a word
cleanedText = re.sub(r'\n',' ',cleanedText)#Get rid of new lines replace with spaces
#Run it 3 times to get most equations but leave most of the text
for x in range(0,2):
    cleanedText = re.sub(r'(\(([^)^(]+)\))','',cleanedText) #removes everything inside of parentheses, have to re-run for nested
    cleanedText = re.sub(r'(\[([^]^[]+)\])','',cleanedText) #removes everything inside of square brackets
    cleanedText = re.sub(r'(\{([^}^{]+)\})','',cleanedText) #removes everything inside of curly brackets 
cleanedText = re.sub(r'[^\w^\s^.]',' ', cleanedText) #Remove all characters not [a-zA-Z0-9_] excluding spaces
cleanedText = re.sub(r'\d','', cleanedText) #Remove all numbers
cleanedText = re.sub(' +', ' ', cleanedText).strip() #Replace all multiple spaces with one space
cleanedText = cleanedText.lower()
cleanedText


'a signiﬁcantly faster algorithm is presented for the original knn mode seeking procedure. it has the advantages over the well known mean shift algorithm that it is feasible in high dimensional vector spaces and results in uniquely well deﬁned modes. moreover without any additional computational eﬀort it may yield a multi scale hierarchy of clusterings. the time complexity is just o. resulting computing times range from seconds for objects to minutes for objects and to less than an hour for objects. the space complexity is just o. the procedure is well suited for ﬁnding large sets of small clusters and is thereby a candidate to analyze thousands of clusters in millions of objects. the knn mode seeking procedure can be used for active learning by assigning the clusters to the class of the modal objects of the clusters. its feasibility is shown by some examples with up to . million handwritten digits. the obtained classiﬁcation results based on the clusterings are compared with those obt

## The Model

# APPENDIX 

## Prelim Model

In [237]:
rough_words = nltk.word_tokenize(cleanedText) 
sent_tokes = nltk.sent_tokenize(cleanedText) 
wordFreqs = {}
sent_scores = {}
stopwords = nltk.corpus.stopwords.words('english')

for w in rough_words:
    if w not in stopwords and re.match(r'[\\A\w]*[aeiou]+[\w$]*',w): 
        if w not in wordFreqs.keys():   
            wordFreqs[w] = 1
        else:
            wordFreqs[w] +=1

mostFreqy = max(wordFreqs.values())

for word in wordFreqs.keys():
    wordFreqs[word] = (wordFreqs[word]/mostFreqy)

        
        
for sentence in sent_tokes:
    #print (sentence)
    for word in sentence.split():
        #print (word)
        if word in wordFreqs.keys():
            #print (word)
            if len(sentence.split(' ')) <30:
                if sentence not in sent_scores.keys():
                    sent_scores[sentence] = wordFreqs[word]
                else:
                    sent_scores[sentence] += wordFreqs[word]
                    
summaryML = heapq.nlargest(10, sent_scores, key = sent_scores.get)

In [227]:
wordFreqs

{'a.': 0.00847457627118644,
 'able': 0.00847457627118644,
 'according': 0.00847457627118644,
 'accuracy': 0.00847457627118644,
 'accurate': 0.01694915254237288,
 'achieved': 0.025423728813559324,
 'acknowledged': 0.00847457627118644,
 'acknowledgments': 0.00847457627118644,
 'active': 0.211864406779661,
 'adding': 0.00847457627118644,
 'addition': 0.00847457627118644,
 'additional': 0.0423728813559322,
 'additionally': 0.00847457627118644,
 'advantage': 0.03389830508474576,
 'advantages': 0.00847457627118644,
 'agglomerative': 0.00847457627118644,
 'ai': 0.025423728813559324,
 'al': 0.06779661016949153,
 'alc': 0.1016949152542373,
 'algorithm': 0.2711864406779661,
 'algorithms': 0.01694915254237288,
 'allr': 0.05084745762711865,
 'almost': 0.03389830508474576,
 'aln': 0.03389830508474576,
 'also': 0.025423728813559324,
 'alternative': 0.00847457627118644,
 'alternatively': 0.00847457627118644,
 'amsterdam': 0.00847457627118644,
 'analysis': 0.00847457627118644,
 'analyze': 0.0084745762

In [239]:
sent_tokes

['a signiﬁcantly faster algorithm is presented for the original knn mode seeking procedure.',
 'it has the advantages over the well known mean shift algorithm that it is feasible in high dimensional vector spaces and results in uniquely well deﬁned modes.',
 'moreover without any additional computational eﬀort it may yield a multi scale hierarchy of clusterings.',
 'the time complexity is just o. resulting computing times range from seconds for objects to minutes for objects and to less than an hour for objects.',
 'the space complexity is just o. the procedure is well suited for ﬁnding large sets of small clusters and is thereby a candidate to analyze thousands of clusters in millions of objects.',
 'the knn mode seeking procedure can be used for active learning by assigning the clusters to the class of the modal objects of the clusters.',
 'its feasibility is shown by some examples with up to .',
 'million handwritten digits.',
 'the obtained classiﬁcation results based on the cluste

In [240]:
sent_scores

{'a big advantage of the proposal is that diﬀerent clustering resolutions related to a set of neighborhood sizes k can be simultaneously computed without the need of any recomputation.': 1.6271186440677963,
 'a drawback of using the clustering for classiﬁcation however is that no classiﬁer is obtained that may be used for out of sample objects.': 1.61864406779661,
 'a fully random choice may result is some very small p cells.': 0.41525423728813565,
 'a number of them those based on ms are deterministic as they are not dependent on a randomly selected subset of objects.': 0.7372881355932204,
 'a randomly selected subset of objects is used for starting the gradient searches.': 1.5847457627118646,
 'a second observation is that the average cluster sizes of the three datasets seem to be diﬀerent.': 1.0084745762711864,
 'a signiﬁcant diﬀerence between the two approaches is the computational eﬀort needed for ﬁnding a series of clusterings with diﬀerent numbers of clusters.': 0.68644067796610

In [238]:
summaryML

['repeat for all n objects xi in s .',
 'knn mode seeking combined with multi level conﬁdence based classiﬁcation in the active learning classiﬁcation procedure as deﬁned above all objects within a cluster receive the same class assignments.',
 'by using the labels of just the modal objects the clustering can be used for labeling all other objects resulting in an active labeling procedure.',
 'cluster evaluation by active learning an important application of clustering of large datasets is active learning representative objects to be labeled are found by the structure in the data.',
 'as on higher clustering levels some clusters may contain objects of diﬀerent lower resolution clusters the object conﬁdences may receive contributions of various classes.',
 'the knn mode seeking procedure can be used for active learning by assigning the clusters to the class of the modal objects of the clusters.',
 's is the user supplied set of objects with size n. .',
 'there is an easy top down proced

## Remove Images and links from PDF's

In [61]:
title = re.search(r'\/.+\.pdf',activePDF).group() #save activePDF's title
title = title[1:]

In [60]:
title

'9905014v1.pdf'

In [69]:
inputStream = open(activePDF, "rb")
outputStream = open("noText.pdf", "wb")

src = PdfFileReader(inputStream)
output = PdfFileWriter()

[output.addPage(src.getPage(i)) for i in range(src.getNumPages())]
#output.removeImages()
#output.removeLinks()

output.write(outputStream)
outputStream.close()

## Sentance Scores

In [23]:
sentence_scores = {}

for sentence in sentence_tokens:
    for word in nltk.word_tokenize(sentence.lower()):
        if word in wordFreqs.keys():
            if len(sentence.split(' ')) <30:
                if sentence not in sentence_scores.keys():
                    sentence_scores[sentence] = wordFreqs[word]
                else:
                    sentence_scores[sentence] += wordFreqs[word]

In [24]:
sentence_scores

{'(This can be reﬁned using a local reward function to express preferences among the diﬀerent states satisfying Ti [8], but we omit this reﬁnement in this paper.)': 2.856643356643357,
 ', Mk}.': 1.6223776223776225,
 ', πn}, one for each subtask.': 2.7377622377622375,
 '.': 3.6503496503496504,
 '1) in which there are two main sub-tasks: Get the passenger (Get) and Deliver the passenger (Put).': 2.9615384615384617,
 '1) to the probability transition function of the recursively optimal policy for j.': 1.5244755244755246,
 '1185–1201, 1994.': 1.6153846153846154,
 '167–173, Morgan Kaufmann, 1993.': 2.6188811188811187,
 '2 The MAXQ Framework  Let M be a Markov decision problem with states S, actions A, reward function R(s′|s, a) and probability transition function P (s′|s, a).': 6.657342657342657,
 '271–278,  San Francisco, CA: Morgan Kaufmann, 1993.': 3.7097902097902096,
 '3 Conditions for Safe State Abstraction  To motivate state abstraction, consider the simple Taxi Task shown in Figure 1

In [25]:
summaryML = heapq.nlargest(10, sentence_scores, key = sentence_scores.get)

In [26]:
summaryML

['There are four special locations in this world, marked as R(ed), B(lue), G(reen), and Y(ellow).',
 '[2] R. S. Sutton, D. Precup, and S. Singh, “Between MDPs and Semi-MDPs: Learning, planning, and representing knowledge at multiple temporal scales,” tech.',
 '[6] M. Hauskrecht, N. Meuleau, C. Boutilier, L. Kaelbling, and T. Dean, “Hierarchical solution of Markov decision processes using macro-actions,” tech.',
 'To learn the values of C(i, x, j) = Q-learning algorithm needs samples of x′ and N drawn according to P (x′, N |x, j).',
 '2 The MAXQ Framework  Let M be a Markov decision problem with states S, actions A, reward function R(s′|s, a) and probability transition function P (s′|s, a).',
 '[9] S. Singh, T. Jaakkola, M. L. Littman, and C. Szpesvari, “Convergence results for single-step on-policy reinforcement-learning algorithms,” tech.',
 'The taxi must go to the passenger’s location (the “source”), pick up the passenger, go to the destination location (the “destination”), and put 

## Get Abstract

In [30]:
corpusAbstract

['This paper presents the MAXQ approach to hierarchical reinforcement learning\nbased on decomposing the target Markov decision process (MDP) into a hierarchy\nof smaller MDPs and decomposing the value function of the target MDP into an\nadditive combination of the value functions of the smaller MDPs. The paper\ndefines the MAXQ hierarchy, proves formal results on its representational\npower, and establishes five conditions for the safe use of state abstractions.\nThe paper presents an online model-free learning algorithm, MAXQ-Q, and proves\nthat it converges wih probability 1 to a kind of locally-optimal policy known\nas a recursively optimal policy, even in the presence of the five kinds of\nstate abstraction. The paper evaluates the MAXQ representation and MAXQ-Q\nthrough a series of experiments in three domains and shows experimentally that\nMAXQ-Q (with state abstractions) converges to a recursively optimal policy much\nfaster than flat Q learning. The fact that MAXQ learns a rep

In [9]:
count =0
for abstract in corpusAbstract:
    corpusAbstract[count] = re.sub(r'\n',' ',abstract)#Get rid of new lines replace with spaces
    

## Removing nested parens takes too much of the document

In [104]:
def remove_nested_parens(input_str):
    """Returns a copy of 'input_str' with any parenthesized text removed. Nested parentheses are handled."""
    result = ''
    paren_level = 0
    for ch in input_str:
        if ch == '(':
            paren_level += 1
        elif (ch == ')') and paren_level:
            paren_level -= 1
        elif not paren_level:
            result += ch
    return result
def remove_nested_brackets(input_str):
    result = ''
    paren_level = 0
    for ch in input_str:
        if ch == '[':
            paren_level += 1
        elif (ch == ']') and paren_level:
            paren_level -= 1
        elif not paren_level:
            result += ch
    return result
def remove_nested_curlybrackets(input_str):
    result = ''
    paren_level = 0
    for ch in input_str:
        if ch == '{':
            paren_level += 1
        elif (ch == '}') and paren_level:
            paren_level -= 1
        elif not paren_level:
            result += ch
    return result

cleanedText = remove_nested_parens(cleanedText)
#cleanedText = remove_nested_brackets(cleanedText)
#cleanedText = remove_nested_curlybrackets(cleanedText)

## PDFMiner 3 - No Need

In [34]:
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io

resource_manager = PDFResourceManager()
fake_file_handle = io.StringIO()
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('C:\\Users\\Al\\Documents\\ByteSizeArxiv\\library/9905014v1.pdf', 'rb') as fh:

    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
        page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

print(text)

9
9
9
1

 

y
a
M
1
2

 

 
 
]

G
L
.
s
c
[
 
 

1
v
4
1
0
5
0
9
9
/
s
c
:
v
i
X
r
a

Hierarchical Reinforcement Learning with the MAXQ Value

Function Decomposition

Thomas G. Dietterich

Department of Computer Science

Oregon State University

Corvallis, OR 97331
tgd@cs.orst.edu

February 1, 2008

Abstract

This paper presents a new approach to hierarchical reinforcement learning based on decomposing
the target Markov decision process (MDP) into a hierarchy of smaller MDPs and decomposing
the value function of the target MDP into an additive combination of the value functions of the
smaller MDPs. The decomposition, known as the MAXQ decomposition, has both a procedural
semantics—as a subroutine hierarchy—and a declarative semantics—as a representation of the
value function of a hierarchical policy. MAXQ uniﬁes and extends previous work on hierar-
chical reinforcement learning by Singh, Kaelbling, and Dayan and Hinton. It is based on the
assumption that the programmer can identify us